In [13]:
import json
import pymongo
from pymongo import UpdateOne
import numpy
import math
from math import radians, cos, sin, asin, sqrt
from collections import Counter
import time
import sys
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import smtplib
import gc
gc.enable() 



def haversine(lng1, lat1, lng2, lat2):
    # 将十进制度数转化为弧度
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    # haversine公式
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlng / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000


b=time.time()
def mailTo(title,mailAdds,message,whoSend='taiwanInfoMaBoss'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()  

mailTo(title="taiwanInfoMaBossbegin",mailAdds=["andy.yuan@wowprime.com"],message="分析開始")
try:
    ######################################################第一次跑用stoneTwo來篩選沒超商的點
    b=time.time()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
#     collectionTaiwan = client.rawData.taiwanInfoStoneTwo
    collectionTaiwan = client.rawData.taiwanInfoMaBoss
    ##################################################第一次跑不執行
    TWLocs=list(collectionTaiwan.find({}, {'_id': False}))
    ##################################################第一次跑不執行
    
    ######################################################IPEEN
    # collection = client.rawData.wowprimeipeen
    collection = client.rawData.ipeenInfo
    ipeendata = list(collection.find({}, {'_id': False}))
    ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
                 and dien['lat'] > 18
                 and dien['lat'] < 27
                 and dien['lng'] < 125
                 and dien['lng'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0
                 and dien['averagecost'] < 250]

    ######################################################IPEEN
    ######################################################conStore0320
    collection = client.rawData.conStore
    conStoreData = list(collection.find({}, {'_id': False}))
    ######################################################conStore0320
    ######################################################Watsons
    collection = client.rawData.Watsons
    watsonsData = list(collection.find({}, {'_id': False}))
    ######################################################Watsons
#     ######################################################carrefour0504
#     collection = client.rawData.carrefour
#     carrefourData = list(collection.find({}, {'_id': False}))
#     ######################################################carrefour0504
    ######################################################pxmart0504
    collection = client.rawData.pxmart
    pxmartData = list(collection.find({}, {'_id': False}))
    ######################################################pxmart0504
#     ######################################################Clinic0531
#     collection = client.rawData.infoClinic
#     clinicData = list(collection.find({}, {'_id': False}))
#     ######################################################Clinic0531
#     ######################################################Tstore0531
#     collection = client.rawData.info3Store
#     tStoreData = list(collection.find({}, {'_id': False}))
#     ######################################################Tstore0531
    ##################################################第一次跑先去除latlng以外的資訊
    TWLocs = [{"lat":i["lat"],"lng":i["lng"]} for i in TWLocs if ('NconStore_Analyze' in i and
              i['NconStore_Analyze'] >0)]
#     TWLocs = [i for i in TWLocs if ('NconStore_Analyze' in i and
#               i['NconStore_Analyze'] >0)]
    ##################################################第一次跑先去除latlng以外的資訊
    allN=len(TWLocs)
    print("有{}筆資料要分析".format(allN))
    radius=500
    n = 0

    for TWLoc in TWLocs:
        score=0
        n += 1
        if n % 50 == 0:
            e=time.time()
            print("已完成{}%分析-{}項,花費{}秒".format(round(n/allN,4)*100,n,round(e-b)))
        if n % 2000 == 0:
            gc.collect()
            client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
            collection = client.rawData.taiwanInfoMaBoss
            
            ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            datas=[i for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,datas)]
            try:
                collection.bulk_write(operations ,ordered=False)
            except Exception as er:
                print(er)
            client.close()

        TWLoc["areaRadius_Analyze"] = radius
        #############################################################0523Score

        TWLoc["NconStore_Analyze"] = len([dien['name'] for dien in conStoreData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= radius])
            
        
        TWLoc["Nwatson_Analyze"] = len([dien for dien in watsonsData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        TWLoc["Npxmart_Analyze"] = len([dien for dien in pxmartData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        
            
#         TWLoc["NclinicData_Analyze"] = len([dien for dien in clinicData
#                                   if haversine(lng1=float(dien["lng"]),
#                                                lat1=float(dien["lat"]),
#                                                lng2=TWLoc["lng"],
#                                                lat2=TWLoc["lat"]) <= radius])
        
#         TWLoc["NtStore_Analyze"] = len([dien for dien in tStoreData
#                                   if haversine(lng1=float(dien["lng"]),
#                                                lat1=float(dien["lat"]),
#                                                lng2=TWLoc["lng"],
#                                                lat2=TWLoc["lat"]) <= radius])

        rangeCost = 1.4

        simDien = [dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['averagecost'] < 200
                   and dien['averagecost'] > 160]
        
        TWLoc["NsimCostDien"] = len(simDien)

        threeMom = len([dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and "三媽" in dien['name']])
        
        TWLoc["NThreeMom_analyze"] = threeMom
        
        tea = len([dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['smallstyle']=='飲料專賣店'])
        TWLoc["NTea_analyze"] = tea
        
        TWLoc["Nmc_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '麥當' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        TWLoc["Nken_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '肯德' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])

        #############################################################0523Score
    for i in TWLocs:
        score=0

        if i['NsimCostDien']>=17:
            score+=3
        elif i['NsimCostDien']>=9:
            score+=2
        elif i['NsimCostDien']>=5:
            score+=1

        if i['NTea_analyze']>=11:
            score+=2
        elif i['NTea_analyze']>=6:
            score+=1

        if i['NThreeMom_analyze']+i["Nmc_Analyze"]+i["Nken_Analyze"]>=1:
            score+=1


        if i['NconStore_Analyze']>=13:
            score+=2
        elif i['NconStore_Analyze']>=5:
            score+=1

        if i['Npxmart_Analyze']+i['Nwatson_Analyze']>=3:
            score+=2
        elif i['Npxmart_Analyze']+i['Nwatson_Analyze']>=1:
            score+=1

        i['score']=score
        
    
    e = time.time()


#     client.close()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collection = client.rawData.taiwanInfoMaBoss
    ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs]
    operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
    try:
        collection.bulk_write(operations ,ordered=False)
    except Exception as er:
        print(er)
    client.close()
    
    e = time.time()
    print(e-b)
    
    
    mailTo(title="taiwanInfoMaBossSuccess",mailAdds=["andy.yuan@wowprime.com"],message="分析完畢_耗時{}秒".format(e-b))
except Exception as er:
    errorMsg=str(er)+"出錯位置:"+str(sys.exc_info()[2].tb_lineno)
    mailTo(title="taiwanInfoMaBossFail",mailAdds=["andy.yuan@wowprime.com"],message=errorMsg)

有50671筆資料要分析
已完成0.1%分析-50項,花費72秒
已完成0.2%分析-100項,花費131秒
已完成0.3%分析-150項,花費188秒
已完成0.38999999999999996%分析-200項,花費245秒
已完成0.49%分析-250項,花費307秒
已完成0.59%分析-300項,花費365秒
已完成0.69%分析-350項,花費425秒
已完成0.79%分析-400項,花費484秒
已完成0.89%分析-450項,花費543秒
已完成0.9900000000000001%分析-500項,花費601秒
已完成1.09%分析-550項,花費659秒
已完成1.18%分析-600項,花費717秒
已完成1.28%分析-650項,花費775秒
已完成1.38%分析-700項,花費833秒
已完成1.48%分析-750項,花費892秒
已完成1.58%分析-800項,花費956秒
已完成1.68%分析-850項,花費1023秒
已完成1.78%分析-900項,花費1083秒
已完成1.87%分析-950項,花費1143秒
已完成1.97%分析-1000項,花費1203秒
已完成2.07%分析-1050項,花費1261秒
已完成2.17%分析-1100項,花費1320秒
已完成2.27%分析-1150項,花費1381秒
已完成2.37%分析-1200項,花費1444秒
已完成2.4699999999999998%分析-1250項,花費1505秒
已完成2.5700000000000003%分析-1300項,花費1568秒
已完成2.6599999999999997%分析-1350項,花費1631秒
已完成2.76%分析-1400項,花費1693秒
已完成2.86%分析-1450項,花費1754秒
已完成2.96%分析-1500項,花費1814秒
已完成3.06%分析-1550項,花費1876秒
已完成3.16%分析-1600項,花費1935秒
已完成3.26%分析-1650項,花費1997秒
已完成3.35%分析-1700項,花費2056秒
已完成3.45%分析-1750項,花費2116秒
已完成3.55%分析-1800項,花費2179秒
已完成3.65%分析-1850項,花費2244秒
已完成3.75%分析-1900項,花費2307秒
已完成3.8

已完成27.83%分析-14100項,花費17115秒
已完成27.93%分析-14150項,花費17172秒
已完成28.02%分析-14200項,花費17230秒
已完成28.12%分析-14250項,花費17288秒
已完成28.22%分析-14300項,花費17347秒
已完成28.32%分析-14350項,花費17405秒
已完成28.42%分析-14400項,花費17463秒
已完成28.52%分析-14450項,花費17522秒
已完成28.62%分析-14500項,花費17580秒
已完成28.71%分析-14550項,花費17638秒
已完成28.810000000000002%分析-14600項,花費17697秒
已完成28.910000000000004%分析-14650項,花費17755秒
已完成29.01%分析-14700項,花費17813秒
已完成29.110000000000003%分析-14750項,花費17871秒
已完成29.21%分析-14800項,花費17930秒
已完成29.310000000000002%分析-14850項,花費17988秒
已完成29.409999999999997%分析-14900項,花費18046秒
已完成29.5%分析-14950項,花費18105秒
已完成29.599999999999998%分析-15000項,花費18163秒
已完成29.7%分析-15050項,花費18221秒
已完成29.799999999999997%分析-15100項,花費18279秒
已完成29.9%分析-15150項,花費18338秒
已完成30.0%分析-15200項,花費18396秒
已完成30.099999999999998%分析-15250項,花費18454秒
已完成30.19%分析-15300項,花費18513秒
已完成30.29%分析-15350項,花費18571秒
已完成30.39%分析-15400項,花費18630秒
已完成30.490000000000002%分析-15450項,花費18688秒
已完成30.59%分析-15500項,花費18745秒
已完成30.69%分析-15550項,花費18803秒
已完成30.79%分析-15600項,花費18862秒
已完成30.89%分析-15650項,

已完成53.879999999999995%分析-27300項,花費32546秒
已完成53.98%分析-27350項,花費32605秒
已完成54.06999999999999%分析-27400項,花費32664秒
已完成54.169999999999995%分析-27450項,花費32722秒
已完成54.269999999999996%分析-27500項,花費32781秒
已完成54.37%分析-27550項,花費32840秒
已完成54.47%分析-27600項,花費32898秒
已完成54.56999999999999%分析-27650項,花費32957秒
已完成54.669999999999995%分析-27700項,花費33014秒
已完成54.769999999999996%分析-27750項,花費33071秒
已完成54.86%分析-27800項,花費33128秒
已完成54.96%分析-27850項,花費33187秒
已完成55.059999999999995%分析-27900項,花費33246秒
已完成55.16%分析-27950項,花費33304秒
已完成55.26%分析-28000項,花費33363秒
已完成55.36%分析-28050項,花費33424秒
已完成55.46%分析-28100項,花費33483秒
已完成55.55%分析-28150項,花費33542秒
已完成55.65%分析-28200項,花費33600秒
已完成55.75%分析-28250項,花費33659秒
已完成55.85%分析-28300項,花費33717秒
已完成55.95%分析-28350項,花費33776秒
已完成56.05%分析-28400項,花費33835秒
已完成56.15%分析-28450項,花費33893秒
已完成56.25%分析-28500項,花費33951秒
已完成56.34%分析-28550項,花費34010秒
已完成56.44%分析-28600項,花費34068秒
已完成56.54%分析-28650項,花費34127秒
已完成56.64%分析-28700項,花費34186秒
已完成56.74%分析-28750項,花費34244秒
已完成56.84%分析-28800項,花費34303秒
已完成56.940000000000005%分析-28850

已完成79.73%分析-40400項,花費47854秒
已完成79.83%分析-40450項,花費47913秒
已完成79.93%分析-40500項,花費47971秒
已完成80.03%分析-40550項,花費48029秒
已完成80.12%分析-40600項,花費48088秒
已完成80.22%分析-40650項,花費48146秒
已完成80.32000000000001%分析-40700項,花費48204秒
已完成80.42%分析-40750項,花費48262秒
已完成80.52%分析-40800項,花費48321秒
已完成80.62%分析-40850項,花費48379秒
已完成80.72%分析-40900項,花費48438秒
已完成80.82000000000001%分析-40950項,花費48496秒
已完成80.91000000000001%分析-41000項,花費48555秒
已完成81.01%分析-41050項,花費48612秒
已完成81.11%分析-41100項,花費48671秒
已完成81.21000000000001%分析-41150項,花費48729秒
已完成81.31%分析-41200項,花費48787秒
已完成81.41000000000001%分析-41250項,花費48845秒
已完成81.51%分析-41300項,花費48904秒
已完成81.6%分析-41350項,花費48962秒
已完成81.69999999999999%分析-41400項,花費49020秒
已完成81.8%分析-41450項,花費49079秒
已完成81.89999999999999%分析-41500項,花費49137秒
已完成82.0%分析-41550項,花費49195秒
已完成82.1%分析-41600項,花費49254秒
已完成82.19999999999999%分析-41650項,花費49312秒
已完成82.3%分析-41700項,花費49370秒
已完成82.39%分析-41750項,花費49428秒
已完成82.49%分析-41800項,花費49487秒
已完成82.59%分析-41850項,花費49545秒
已完成82.69%分析-41900項,花費49603秒
已完成82.78999999999999%分析-41950項,花費49662秒
已

In [20]:
import pandas

In [43]:
df=pandas.DataFrame(TWLocs)

In [44]:
writer=pandas.ExcelWriter(r"D:\outputXLS\maBoss.xlsx")

In [45]:
sa=df.to_excel(writer)

In [46]:
writer.save()

In [75]:
import pymongo
from pymongo import UpdateOne
client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
collection = client.rawData.taiwanInfoMaBoss


TWLocs=list(collection.find({},{"_id":False}))

In [82]:
for i in TWLocs:
    score=0
    
    if i['NsimCostDien']>=17:
        score+=3
    elif i['NsimCostDien']>=9:
        score+=2
    elif i['NsimCostDien']>=5:
        score+=1
    
    if i['NTea_analyze']>=11:
        score+=2
    elif i['NTea_analyze']>=6:
        score+=1
    
    if i['NThreeMom_analyze']+i["Nmc_Analyze"]+i["Nken_Analyze"]>=1:
        score+=1
    

    if i['NconStore_Analyze']>=13:
        score+=2
    elif i['NconStore_Analyze']>=5:
        score+=1

    if i['Npxmart_Analyze']+i['Nwatson_Analyze']>=3:
        score+=2
    elif i['Npxmart_Analyze']+i['Nwatson_Analyze']>=1:
        score+=1

    i['score']=score
    
        
    

In [83]:
TWLocs[0]

{'NTea_analyze': 0,
 'NThreeMom_analyze': 0,
 'NconStore_Analyze': 1,
 'Nken_Analyze': 0,
 'Nmc_Analyze': 0,
 'Npxmart_Analyze': 0,
 'NsimCostDien': 0,
 'Nwatson_Analyze': 0,
 'areaRadius_Analyze': 500,
 'lat': 21.93315505999999,
 'lng': 120.827842325001,
 'score': 0}

In [85]:
# client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
collection = client.rawData.taiwanInfoMaBoss
ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs]
operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
try:
    collection.bulk_write(operations ,ordered=False)
except Exception as er:
    print(er)
client.close()